## Setup and Model Loading

In [1]:
# Import required libraries
from transformers import pipeline
import torch
import pandas as pd
from tqdm.auto import tqdm
import json
import re
import os
import gc
import ast
import signal
import sys
from io import StringIO
from collections import defaultdict
import random
from typing import List, Dict, Tuple, Optional
from datasets import Dataset

/home/nafi/dev/shared-task/task2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Memory management and system check
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

clear_memory()

# System resources
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {gpu_memory:.1f} GB")
else:
    print("No GPU detected - using CPU (will be very slow)")

GPU: NVIDIA GeForce RTX 3090 Ti
GPU Memory: 22.0 GB


## Data Loading

In [3]:
dev_data_path = 'PATH_TO_ORIGINAL_DEV_DATASET'
dev_df = pd.read_csv(dev_data_path)

In [4]:
pipe = pipeline(
        "text-generation", 
        model="unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit",
        trust_remote_code=True,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]
Device set to use cuda:0


In [5]:
def format_prompt(example):
        """Format a single example into the required prompt format"""
        instruction = example['instruction']
        test_list = example['test_list']
        
        # Parse function name from instruction
        function_name = "unknown_function"
        if "Example:" in instruction:
            example_part = instruction.split("Example:")[1].strip()
            func_match = re.search(r'(\w+)\s*\(', example_part)
            if func_match:
                function_name = func_match.group(1)
        
       
        system_message = """You are an expert Python programmer. Your task is to generate clean, efficient, and correct Python functions that pass all given test cases.

CRITICAL RULES:
1. ALWAYS wrap your code in ```python ``` blocks
2. Write ONLY the function implementation, no extra explanations
3. Use the EXACT function name from the example
4. Ensure the function passes ALL test cases
5. Handle edge cases and invalid inputs appropriately
6. Use appropriate data types based on test case patterns

Here are examples of how to solve different types of problems:

EXAMPLE 1 - String Processing:
Task: একটি প্রদত্ত স্ট্রিং-এ প্রথম পুনরাবৃত্ত অক্ষর খুঁজে পেতে একটি পাইথন ফাংশন লিখুন।
Test Cases:
assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == "None"  
assert first_repeated_char("123123") == "1"

Expected Solution:
```python
def first_repeated_char(s):
    seen = set()
    for char in s:
        if char in seen:
            return char
        seen.add(char)
    return "None"
```

EXAMPLE 2 - Mathematical Function:
Task: প্রদত্ত পূর্ণসংখ্যাটি একটি মৌলিক সংখ্যা কিনা তা পরীক্ষা করার জন্য একটি ফাংশন লিখুন।
Test Cases:
assert prime_num(13) == True
assert prime_num(7) == True
assert prime_num(-1010) == False

Expected Solution:
```python
def prime_num(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True
```


Code Quality Standards:
- Write code with proper indentation
- Optimize for correctness first, then efficiency
- Handle common edge cases (empty inputs, None values, negative numbers, etc.)
- Return the exact data type shown in test cases"""
        
        user_prompt = f"""Generate a Python function for this problem:

**Task**: {instruction}

**Test Cases**:
{test_list}

**Expected Function Name**: {function_name}

Requirements:
- Follow the examples shown in the system message
- Analyze the test cases carefully to understand input/output patterns
- Implement the function to pass ALL test cases exactly
- Return the appropriate data type as shown in test cases
- Handle edge cases gracefully (empty inputs, invalid values, etc.)
- Use efficient algorithms where applicable

Generate ONLY the Python function wrapped in ```python ``` blocks. No explanations needed."""
        
        # Format for chat template
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]
        
        # Apply chat template
        formatted_prompt = pipe.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        return formatted_prompt

In [6]:
instructions_data = []
formatted_prompts = []
ids_list = []
for _, row in tqdm(dev_df.iterrows(), desc="Preparing data", unit="row", total=len(dev_df)):
    instructions_data.append({
        'instruction': row['instruction'],
        'test_list': row['test_list'],
        'id': row['id']
    })

for item in tqdm(instructions_data, desc="Formatting prompts", unit="prompt"):
    formatted_prompt = format_prompt(item)
    formatted_prompts.append(formatted_prompt)
    ids_list.append(item['id'])

print(f"Formatted {len(formatted_prompts)} prompts")
print("Creating dataset from formatted prompts...")

dataset_dict = {
    'prompt': formatted_prompts,
    'id': ids_list
}

dataset = Dataset.from_dict(dataset_dict)
print(f"Dataset created with {len(dataset)} samples")

Formatting prompts: 100%|██████████| 400/400 [00:00<00:00, 30568.50prompt/s]

Formatted 400 prompts
Creating dataset from formatted prompts...
Dataset created with 400 samples


## Simple Code Generation

In [7]:
def generate_code(prompt):
    """
    Generate code from the given prompt using the language model pipeline.
    Returns: (generated_code)
    """
    
    result = pipe(
                prompt,
                max_new_tokens=768,
                temperature=0.1,
                top_p=0.95,
                do_sample=True,
                return_full_text=False,
                pad_token_id=pipe.tokenizer.eos_token_id
            )
            
    generated_code = result[0]['generated_text'].strip()
    
    return generated_code

In [8]:
responses = []

for idx in tqdm(range(len(dataset)), desc="Code generation"):
    try:
        # Get sample data
        prompt = dataset[idx]['prompt']
        sample_id = dataset[idx]['id']
        
        # Get corresponding test cases and instruction from original data
        original_row = dev_df[dev_df['id'] == sample_id].iloc[0]
        test_cases_str = original_row['test_list']
        instruction = original_row['instruction']
        
        # Generate code
        generated_code = generate_code(prompt)
        
        responses.append(generated_code)   
        
        # Memory management - clear every 20 samples
        if (idx + 1) % 20 == 0:
            clear_memory()
            print(f"\nMemory cleared after {idx + 1} samples")
            
        # Progress update every 50 samples
        if (idx + 1) % 50 == 0:
            print(f"\nProgress Update after {idx + 1} samples:") 
            
    except Exception as e:
        print(f"Complete failure for ID {sample_id}: {e}")
        responses.append("def placeholder(): pass")
        continue


print("CODE GENERATION COMPLETED!")


Code generation:   5%|▌         | 20/400 [00:58<20:08,  3.18s/it]


Memory cleared after 20 samples


Code generation:  10%|█         | 40/400 [01:46<15:06,  2.52s/it]


Memory cleared after 40 samples


Code generation:  12%|█▎        | 50/400 [02:08<13:12,  2.26s/it]

...


Memory cleared after 360 samples


Code generation:  95%|█████████▌| 380/400 [16:57<00:54,  2.75s/it]


Memory cleared after 380 samples


Code generation: 100%|██████████| 400/400 [17:52<00:00,  2.68s/it]


Memory cleared after 400 samples

Progress Update after 400 samples:
CODE GENERATION COMPLETED!


In [9]:
submission_data = []
for i, (_, row) in enumerate(dev_df.iterrows()):
    submission_data.append({
        "id": int(row['id']),
        "response": responses[i]
    })

submission_file = "submission.json"
with open(submission_file, 'w', encoding='utf-8') as f:
    json.dump(submission_data, f, ensure_ascii=False, indent=2)


print("SUBMISSION SAVED!")
print(f"File: {submission_file}")
print(f"Total samples: {len(submission_data)}")

SUBMISSION SAVED!
File: submission.json
Total samples: 400
